In [ ]:
# Importing neccessary libraries and methods

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
import copy

In [ ]:
#loading X_train and y_train data

In [2]:
X_train=pd.read_csv("X_train.csv")
y_train=pd.read_csv("y_train.csv")

In [3]:
print('Shape of training features',X_train.shape)
print('Shape of training target',y_train.shape)

Shape of training features (33050, 44)
Shape of training target (33050, 2)


In [4]:
# Since RandomForest show the most promise during my testing of different machine learning algorithms I have chosen 
#it as my final on which i will try hyptertuning its paramteres to squeeze out more performance

In [5]:
column_deletion_threshold=[i for i in range(1,100) if i%20==0]
column_deletion_threshold

[20, 40, 60, 80]

In [6]:
row_deletion_threshold=[i for i in range(1,100) if i%20==0]
row_deletion_threshold

[20, 40, 60, 80]

In [7]:
imputing_strategy_numerical=['mean','median']
# there is no imputing strategy for our categorical data as they have no NaN values

In [8]:
## Tuning Parameteres of Random Forest.( Parameters range choosen using the best tpot model)

In [9]:
n_estimators_search=[i for i in range(90,101) if i%5==0]
n_estimators_search

[90, 95, 100]

In [10]:
criterion_search=['gini', 'entropy']

In [11]:
max_features_search=[i for i in range(13,16)]   
max_features_search

[13, 14, 15]

In [12]:
min_samples_leaf_search=[i for i in range(4,7)]
min_samples_leaf_search

[4, 5, 6]

In [13]:
min_samples_split_search=[i for i in range(8,12)]
min_samples_split_search

[8, 9, 10, 11]

In [14]:
# Since we are using random forest we dont need to normalize/scale the data


# using roc_curve_score can be a bit misleading in cases where the number of examples in each predictor classes are 
#imblanced

In [ ]:
#Creating placeholder variables to store the best value from the parameter search

In [15]:
best_model_parameters={}
best_roc_curve_score=0

In [ ]:
# creating parameter grid to pass onto to the classifier using HalvingGridSearchCV

In [16]:
parameter_grid={'n_estimators':n_estimators_search,'criterion':criterion_search,
                'min_samples_split':min_samples_split_search,'min_samples_leaf':min_samples_leaf_search,
                'max_features':max_features_search
               }

In [17]:
parameter_grid

{'n_estimators': [90, 95, 100],
 'criterion': ['gini', 'entropy'],
 'min_samples_split': [8, 9, 10, 11],
 'min_samples_leaf': [4, 5, 6],
 'max_features': [13, 14, 15]}

In [18]:
X_train.head()

,Unique_ID,C1,C2,C3,C4,C5,C6,C7,C8,N1,...,N26,N27,N28,N29,N30,N31,N32,N33,N34,N35
0,Candidate_5926,1,0,11,31,0,False,0,True,23.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,113.39,12.0
1,Candidate_48134,1,4,2,66,2,False,1,True,11.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,262.10,17.0
2,Candidate_51717,1,0,19,2,0,False,0,True,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,50.29,18.0
3,Candidate_26401,1,1,16,47,1,False,4,True,17.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,126.52,27.0
4,Candidate_34872,1,1,13,1,1,True,6,True,27.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,205.47,21.0


In [19]:
y_train.head()

,Unique_ID,Dependent_Variable
0,Candidate_5926,1
1,Candidate_48134,0
2,Candidate_51717,1
3,Candidate_26401,0
4,Candidate_34872,0


In [ ]:
#Converting boolean values into integers

In [20]:
X_train['C6']=X_train['C6'].astype(int)
X_train['C8']=X_train['C6'].astype(int)

In [ ]:
# function to clean data and impute missing values accordingly.
# The data is split into training and validation set 
# The hyperparameters are passed on as a list

In [21]:
def process_clean_data(col_thres,row_thres,impute_strat):
    drop_col_names=[]
    col_nan_count_threshold=(col_thres*X_train.shape[0])/100
    col_nan_count_threshold=int(col_nan_count_threshold)
    X=X_train.dropna( axis=1, 
                thresh=col_nan_count_threshold)
    
    for col in X_train.columns:
        if col not in X.columns:
            drop_col_names.append(col)
            
    
    
    
    
    
    row_nan_count_threshold=int((row_thres*X_train.shape[1])/100)
    X=X.dropna(axis=0,thresh=row_nan_count_threshold)
    col_elim=[]
   
    
   
    for col in X.columns:
        if X[col].isna().sum()>0:
            if impute_strat=='mean':
                imp=X[col].mean()
                
            else:
                imp=X[col].median()
            X[col]=X[col].fillna(imp)
            col_elim.append([col,imp])
            

    
    y=y_train.loc[X.index]
    

   
    X=X.drop(['Unique_ID'],axis=1)
    y=y.drop(['Unique_ID'],axis=1)
    
    assert X.isnull().values.any() == False
    assert y.isnull().values.any() == False
     
    
    X= X.to_numpy(dtype='float64')
    y=y.to_numpy(dtype='float64')
    y=y.ravel()
    
        
    
    X_train_split,X_val,y_train_split,y_val=train_test_split(X,y,test_size=0.2,random_state=42)
    
 
    return X_train_split,X_val,y_train_split,y_val,col_elim,drop_col_names
        
    
    
    
    
    
    

 
    
    
    


In [ ]:
#Creating a loop over loop to search for the best hyperparameters and impute strategy

In [23]:
for column_deletion in column_deletion_threshold:
    for row_deletion in row_deletion_threshold:
        for impute_strategy in imputing_strategy_numerical:
            X_train_split,X_val,y_train_split,y_val,col_elim,drop_col_names=process_clean_data(column_deletion,
                                                                                row_deletion,impute_strategy)
            random_forest_model=RandomForestClassifier(random_state=42)
            grid_search=HalvingGridSearchCV(random_forest_model,parameter_grid,scoring='roc_auc',n_jobs=-1
                                           ,random_state=42,cv=3,verbose=1,aggressive_elimination=True)
            best_estimator=grid_search.fit(X_train_split,y_train_split)
            y_pred=best_estimator.predict(X_val)
            print(accuracy_score(y_true=y_val,y_pred=y_pred))
            print(classification_report(y_val,y_pred)) 
            # creating a confusion matrix 
            print(confusion_matrix(y_val, y_pred) )
            auc_score= roc_auc_score(y_val,y_pred)
            print('auc score',auc_score)
            if auc_score> best_roc_curve_score:
                print('here')
                best_model=grid_search.best_estimator_
                best_roc_curve_score=auc_score
                best_model_parameters=copy.deepcopy(grid_search.best_params_)
                best_model_parameters['column_deletion_threshold']=column_deletion
                best_model_parameters['row_deletion_threshold']=row_deletion
                best_model_parameters['impute_strategy_numerical']=impute_strategy
                best_model_parameters['impute_column_names']=col_elim
                best_model_parameters['drop_col_names']=drop_col_names
                print(best_model_parameters)
                
                
                
        

                                     
                                     
            
            
            

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 326
max_resources_: 26440
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 216
n_resources: 326
Fitting 3 folds for each of 216 candidates, totalling 648 fits
----------
iter: 1
n_candidates: 72
n_resources: 978
Fitting 3 folds for each of 72 candidates, totalling 216 fits
----------
iter: 2
n_candidates: 24
n_resources: 2934
Fitting 3 folds for each of 24 candidates, totalling 72 fits
----------
iter: 3
n_candidates: 8
n_resources: 8802
Fitting 3 folds for each of 8 candidates, totalling 24 fits
----------
iter: 4
n_candidates: 3
n_resources: 26406
Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.7444780635400907
              precision    recall  f1-score   support

         0.0       0.76      0.92      0.83      4595
         1.0       0.65      0.35      0.45      2015

    accuracy                           0.74      6610
   macro avg       0.71      0.63      0.64

----------
iter: 4
n_candidates: 3
n_resources: 22680
Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.7360182905381639
              precision    recall  f1-score   support

         0.0       0.75      0.94      0.83      4007
         1.0       0.63      0.26      0.37      1679

    accuracy                           0.74      5686
   macro avg       0.69      0.60      0.60      5686
weighted avg       0.71      0.74      0.70      5686

[[3753  254]
 [1247  432]]
auc score 0.5969534702002288
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 280
max_resources_: 22743
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 216
n_resources: 280
Fitting 3 folds for each of 216 candidates, totalling 648 fits
----------
iter: 1
n_candidates: 72
n_resources: 840
Fitting 3 folds for each of 72 candidates, totalling 216 fits
----------
iter: 2
n_candidates: 24
n_resources: 2520
Fitting 3 folds for each of 24 candidates, totalling

----------
iter: 1
n_candidates: 72
n_resources: 840
Fitting 3 folds for each of 72 candidates, totalling 216 fits
----------
iter: 2
n_candidates: 24
n_resources: 2520
Fitting 3 folds for each of 24 candidates, totalling 72 fits
----------
iter: 3
n_candidates: 8
n_resources: 7560
Fitting 3 folds for each of 8 candidates, totalling 24 fits
----------
iter: 4
n_candidates: 3
n_resources: 22680
Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.7360182905381639
              precision    recall  f1-score   support

         0.0       0.75      0.94      0.83      4007
         1.0       0.63      0.26      0.37      1679

    accuracy                           0.74      5686
   macro avg       0.69      0.60      0.60      5686
weighted avg       0.71      0.74      0.70      5686

[[3753  254]
 [1247  432]]
auc score 0.5969534702002288
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 280
max_resources_: 22743
aggressive_elimination: True
facto

----------
iter: 4
n_candidates: 3
n_resources: 22761
Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.7378862359550562
              precision    recall  f1-score   support

         0.0       0.75      0.94      0.83      4007
         1.0       0.64      0.27      0.38      1689

    accuracy                           0.74      5696
   macro avg       0.70      0.60      0.61      5696
weighted avg       0.72      0.74      0.70      5696

[[3750  257]
 [1236  453]]
auc score 0.6020341400772449
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 280
max_resources_: 22743
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 216
n_resources: 280
Fitting 3 folds for each of 216 candidates, totalling 648 fits
----------
iter: 1
n_candidates: 72
n_resources: 840
Fitting 3 folds for each of 72 candidates, totalling 216 fits
----------
iter: 2
n_candidates: 24
n_resources: 2520
Fitting 3 folds for each of 24 candidates, totalling

----------
iter: 1
n_candidates: 72
n_resources: 843
Fitting 3 folds for each of 72 candidates, totalling 216 fits
----------
iter: 2
n_candidates: 24
n_resources: 2529
Fitting 3 folds for each of 24 candidates, totalling 72 fits
----------
iter: 3
n_candidates: 8
n_resources: 7587
Fitting 3 folds for each of 8 candidates, totalling 24 fits
----------
iter: 4
n_candidates: 3
n_resources: 22761
Fitting 3 folds for each of 3 candidates, totalling 9 fits
0.7378862359550562
              precision    recall  f1-score   support

         0.0       0.75      0.94      0.83      4007
         1.0       0.64      0.27      0.38      1689

    accuracy                           0.74      5696
   macro avg       0.70      0.60      0.61      5696
weighted avg       0.72      0.74      0.70      5696

[[3750  257]
 [1236  453]]
auc score 0.6020341400772449
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 280
max_resources_: 22743
aggressive_elimination: True
facto

In [ ]:
#printing the best roc_curve_score from all models fit

In [24]:
best_roc_curve_score

0.633000861331094

In [ ]:
#printing best parameters value

In [25]:
best_model_parameters

{'criterion': 'gini',
 'max_features': 14,
 'min_samples_leaf': 6,
 'min_samples_split': 11,
 'n_estimators': 100,
 'column_deletion_threshold': 20,
 'row_deletion_threshold': 20,
 'impute_strategy_numerical': 'mean',
 'impute_column_names': [['N2', 81.34271045783811],
  ['N3', 3.359360562949365],
  ['N4', 9.574075376177753],
  ['N5', 8.35012656447757],
  ['N6', 3.45436056294974],
  ['N7', 25.229929804125923],
  ['N10', 0.9033810501793214],
  ['N10.1', 2.044048677313552],
  ['N11', 7.108678171584091],
  ['N12', 1016.0185680123786],
  ['N14', 4.537759694223538],
  ['N15', 0.3301351807007326],
  ['N16', 0.02320348755449304],
  ['N17', 15820.726796512445],
  ['N18', 0.5289069751089813],
  ['N19', 11037.365321816074],
  ['N20', 21.764141999367954],
  ['N21', 0.857198637592601],
  ['N22', 0.906422276063064],
  ['N23', 58.4333627797409],
  ['N35', 19.58725439107378]],
 'drop_col_names': ['N25', 'N26', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32']}

In [28]:
#printing the best model

In [92]:
best_model

RandomForestClassifier(max_features=14, min_samples_leaf=6,
                       min_samples_split=11, random_state=42)

In [93]:
#Loading X_test 

In [99]:
X_test=pd.read_csv('X_test.csv')

In [100]:
X_test.describe()

,C1,C2,C3,C4,C5,C7,N1,N2,N3,N4,...,N26,N27,N28,N29,N30,N31,N32,N33,N34,N35
count,11017.000000,11017.000000,11017.000000,11017.000000,11017.000000,11017.000000,11017.000000,9528.000000,10905.000000,9529.000000,...,2138.000000,2138.000000,2138.000000,2138.000000,2138.000000,2138.000000,2112.000000,11017.000000,11017.000000,10888.000000
mean,1.125533,2.463647,12.883997,14.488427,1.735046,3.117636,20.092037,81.279492,3.360064,9.529961,...,20.024790,19.379326,0.577643,0.066885,6995.496043,2180.824972,1.303504,125.209108,223.843161,19.606539
std,0.409512,3.409954,11.964840,15.628293,1.324165,2.133982,8.149355,83.802131,0.404097,5.533917,...,15.556284,15.135256,2.147313,0.762798,6247.803063,3105.571653,49.867054,104.491638,185.433746,7.332430
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-164.000000,20.000000,0.000000,3.000000
25%,1.000000,0.000000,2.000000,1.000000,1.000000,2.000000,13.500000,21.000000,3.200000,6.000000,...,9.000000,9.000000,0.000000,0.000000,3000.000000,0.000000,-32.000000,54.000000,98.740000,15.000000
50%,1.000000,1.000000,10.000000,10.000000,2.000000,4.000000,19.500000,53.000000,3.400000,9.000000,...,15.000000,15.000000,0.000000,0.000000,5000.000000,872.345000,0.000000,90.000000,172.760000,19.000000
75%,1.000000,3.000000,20.000000,24.000000,2.000000,4.000000,26.990000,114.000000,3.600000,13.000000,...,29.000000,28.000000,0.000000,0.000000,9000.000000,3303.902500,30.000000,160.000000,302.140000,24.000000
max,3.000000,20.000000,51.000000,67.000000,7.000000,7.000000,49.750000,551.000000,4.400000,56.000000,...,116.000000,114.000000,40.000000,21.000000,53200.000000,21723.490000,221.000000,700.000000,2251.510000,57.000000


In [101]:
#Using the best model to predict X_test. Also imputing missing values found best during training 

In [102]:
def predict_x_test(best_model,best_model_parameters):
    X_test['C6']=X_train['C6'].astype(int)
    X_test['C8']=X_train['C6'].astype(int)
    
    drop_col=best_model_parameters['drop_col_names']
    X_test.drop(drop_col,axis=1,inplace=True)
    
    for col,value in best_model_parameters['impute_column_names']:
        if X_test[col].isna().any():
            X_test[col]=X_test[col].fillna(value)
    unique_id=X_test['Unique_ID']
    X_test.drop(['Unique_ID'],axis=1,inplace=True)
    X_test.to_numpy(dtype='float64')
    
    y_pred=best_model.predict_proba(X_test)
    y_pred=y_pred[:,1]
    
    y_pred=pd.DataFrame(data=y_pred,columns=['Class_1_Probability'])
    y_pred['Unique_ID']=unique_id
    y_pred=y_pred[['Unique_ID','Class_1_Probability']]
    print(y_pred.head())
    
    
    y_pred.to_csv('final_predictions.csv',index=False)
    
    
        

In [103]:
predict_x_test(best_model,best_model_parameters)

         Unique_ID  Class_1_Probability
0   Candidate_1602             0.396987
1  Candidate_29650             0.268796
2  Candidate_31061             0.316141
3   Candidate_5768             0.150551
4  Candidate_27059             0.528099
